<a href="https://colab.research.google.com/github/TranTrungHa/Bankrrupcy_AI/blob/main/fuzz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import gc
import os
import pandas as pd, datetime as dt
from datetime import date
import regex as re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

list_look_u = {'MA_KHACH_HANG': 'Mã KH aft_clean', 'CARD_NUMBER':'Số thẻ aft_clean', 
               'NGAY_PHAT_HANH': 'ngày tháng aft_clean', 'TEN_KHACH_HANG':'Họ và Tên aft_clean',
               'LOAI_SAN_PHAM':'Loại thẻ aft_clean'}

list_pair_1 = ['Họ và Tên aft_clean_x',  'Số thẻ aft_clean_x',
                'Loại thẻ aft_clean_x','ngày tháng aft_clean_x', 'Mã Chi nhánh aft_clean_x']
list_pair_2 = ['Họ và Tên aft_clean_y' , 'Số thẻ aft_clean_y' ,
               'Loại thẻ aft_clean_y','ngày tháng aft_clean_y' ,'Mã Chi nhánh aft_clean_y']
list_pair_3 = {'Họ và Tên aft_clean_x' : 'Họ và Tên aft_clean_y',  'Số thẻ aft_clean_x' : 'Số thẻ aft_clean_y',
                'Loại thẻ aft_clean_x' : 'Loại thẻ aft_clean_y' ,'ngày tháng aft_clean_x' : 'ngày tháng aft_clean_y'
                , 'Mã Chi nhánh aft_clean_x': 'Mã Chi nhánh aft_clean_y'}

# Đặt tên File cuối cùng theo ngày xuất
Name_File =  "C:/Users/ha.tt6/Desktop/PhatFullControll/HuyThe/FileSauCung{}.xlsx".format(date.today().strftime("%d%m%Y"))

# Lấy file so sánh
#File_Full = pd.read_excel(r'E:/BaK_F/Data_123/old/Tổng_Phát_Hành.xlsx'
#                    ,dtype={'MA_KHACH_HANG':str,'Trang thai':str},sheet_name="Process",header =0
#                    ,na_values ='N/A', usecols ='A,B,E,F,G,H,I,J,K,L,M,P,Q,R')
df_all = pd.read_excel('/content/Tdata.xlsx',header =0,na_values ='N/A')
File_CP = pd.read_excel('/content/drive/MyDrive/Card_portal_05.xlsx'
                    ,dtype={'Mã khách hàng':str},sheet_name="Sheet1",header =0,na_values ='N/A')

#Xử lý file so sánh
#Số thẻ nếu re dạng số thẻ: 4 số đầu, 4 số cuối và *** hoặc xxx
Sothe = '(\d{4})(\*{1,16})(\d{4})|(\d{4})(x{1,16})(\d{4})'
#Loại thẻ nếu nằm trong list loại thẻ
Loaithe = '(VS)|(VISA)|(MC)|(LOCAL)'
#ngày phát hànhnếu chứa số và / , len =< 10
ngaythang = '(\d+/\d+/\d+)|(\d+-\d+-\d+)'
#ID Nếu chỉ chứa số và hay bắt đầu với 1
ID = '^[1][0-9]{0,8}'
#Tên nếu chỉ chứa chữ và độ unique lớn:
HoTen = '[a-zA-Z\s]+$'
#MÃ DVKD bắt đầu với VN001
MACN = '^VN001'
# List tên hình mẫu và hình mẫu tương ứng
list_partten = {'Số thẻ std':Sothe, 'Loại thẻ std':Loaithe, 'ngày tháng std':ngaythang, 'Mã KH std':ID,'Mã Chi nhánh std':MACN}
# Xác định cột chứa số thẻ, loại thẻ, ngày phát hành, ID, Tên KH, Mã DVKD
def find_col(data_in, pattern):
  data1 = data_in
  len1 = len(data1.index)
  for name1 in data1.columns:
    for name_pat, pat in pattern.items():
      counting1 = (data1[name1].astype(str).str.count(pat, flags=2).sum())/len1
      if counting1 > 0.8:
        data1[name_pat] = data1[name1]
        print('detect column {}'.format(name1))
  data3 = clean_data(data1)
  return data3

def find_col_name(data_in, pattern):
  data2 = data_in
  for name1 in data2.columns:
    for name_pat, pat in pattern.items():
      if name1 not in name_pat:
        if ((data2[name1].astype(str).str.count(HoTen).sum())/len(data2) > 0.85) & (data2[name1].nunique()/len(data2) > 0.9):
          try:
            data2['Họ và Tên aft_clean'] = data2[name1].str.strip()
            print('Họ và Tên is cleaned')
            break
          except:
            print('Cant clean Họ và Tên')
  return data2


def clean_data(data_in2):
  data_cleaned = data_in2
  try:
    data_cleaned['Số thẻ aft_clean'] = data_cleaned['Số thẻ std'].str[:4] + 'xxxxxx' + data_cleaned['Số thẻ std'].str[-4:]
    print('Số thẻ is cleaned')
  except:
    print('Cant clean Số thẻ')
  try:
    data_cleaned['Loại thẻ aft_clean'] = data_cleaned['Loại thẻ std'].str.replace('( Ecommer)|( Main)|( Sub)','')
    print('Loại thẻ is cleaned')
  except:
    print('cant clean loại thẻ')
  try:
    data_cleaned['ngày tháng aft_clean'] = pd.to_datetime(data_cleaned['ngày tháng std'])
    print('ngày tháng is cleaned')
  except:
    print('cant clean ngày tháng')
  try:
    data_cleaned['Mã KH aft_clean'] = data_cleaned['Mã KH std'].astype(str)
    print('Mã KH is cleaned')
  except:
    print('cant clean Mã KH')
  try:
    data_cleaned['Mã Chi nhánh aft_clean'] = data_cleaned['Mã Chi nhánh std']
    print('Mã Chi nhánh is cleaned')
  except:
    print('cant clean Mã Chi nhánh')
  return data_cleaned


File_CP['Số thẻ std'] = File_CP['Số thẻ']
File_CP['Loại thẻ std'] = File_CP['Loại sản phẩm']
File_CP['ngày tháng std'] = File_CP['Ngày phát hành']
File_CP['Mã KH std'] = File_CP['Mã khách hàng']
File_CP['Mã Chi nhánh std'] = File_CP['Mã CN']
File_CP['Họ và Tên aft_clean'] = File_CP['Tên khách hàng']


#Fuzzywuzzy: Tìm kiếm ID match, trả lại bảng
#Sử dụng fuzz để tính điểm cho số thẻ, tên khách hàng, ngày phát hành, loại sản phẩm
#trả lại bảng

    
ketqua1 = find_col(df_all,list_partten)
ketqua2 = find_col_name(ketqua1, list_partten)
ketqua3 = clean_data(File_CP)
#saucung = saucung.drop(columns=['Số thẻ std', 'Loại thẻ std', 'ngày tháng std', 'Mã KH std','Mã Chi nhánh std'])


detect column loai the
detect column so the
detect column ngay
detect column id
detect column MA_CN
Số thẻ is cleaned
Loại thẻ is cleaned
ngày tháng is cleaned
Mã KH is cleaned
Mã Chi nhánh is cleaned
Họ và Tên is cleaned


<ipython-input-5-3ddbd23407e4>:83: FutureWarning: The default value of regex will change from True to False in a future version.
  data_cleaned['Loại thẻ aft_clean'] = data_cleaned['Loại thẻ std'].str.replace('( Ecommer)|( Main)|( Sub)','')
<ipython-input-5-3ddbd23407e4>:88: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data_cleaned['ngày tháng aft_clean'] = pd.to_datetime(data_cleaned['ngày tháng std'])


Số thẻ is cleaned


<ipython-input-5-3ddbd23407e4>:83: FutureWarning: The default value of regex will change from True to False in a future version.
  data_cleaned['Loại thẻ aft_clean'] = data_cleaned['Loại thẻ std'].str.replace('( Ecommer)|( Main)|( Sub)','')


Loại thẻ is cleaned
ngày tháng is cleaned
Mã KH is cleaned
Mã Chi nhánh is cleaned


In [23]:

def fuzzy_match(val1, val2):
  return fuzz.partial_ratio(str(val1), str(val2))

def sosanh_percent(data_input,data_CP):
    data_temp = data_input
    Data1 = data_temp.merge(data_CP, on='Mã KH aft_clean', how='left')
    for keyy, valuee in list_pair_3.items():
      Data1['fuzzyw'+str(keyy)+str(valuee)] = Data1.apply(lambda row: fuzzy_match(row[keyy], row[valuee]), axis=1)
    Data1['WuzzySum'] = 0
    for col in Data1.columns:
      if col[:6] == 'fuzzyw':
        Data1['WuzzySum'] += Data1[col]
    Data1['WuzzySum'] = Data1['WuzzySum']/len(list_pair_1)
    return Data1
    
saucung = sosanh_percent(ketqua2,ketqua3)
saucung.to_excel('drive/MyDrive/ketquasc.xlsx')

In [2]:
pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
